In [9]:
my_API_Key=''
my_API_Secret=''

In [10]:
# Importing 'binance' to access the price data
from binance import Client
# Importing pandas to manage the data
import pandas as pd
# Importing datetime to manage the date and time of the prices
import datetime
# Importing pyplot to plot the prices 
from matplotlib import pyplot as plt
# Importing ta library
import ta
import pandas_ta as pta

In [11]:
# Creating a function to retrieve the price data from binance
def give(shareName,timeframe,limit):
    symbol=shareName+'USDT' #for example, 'BTCUSDT'
    API_Key=my_API_Key #Typed above
    API_Secret=my_API_Secret #Typed above
    client = Client(API_Key, API_Secret)
    klines = client.get_historical_klines(symbol=symbol, interval=timeframe, limit=limit)
    return klines

In [12]:
# Creating a function to handle the retrieved data as a dataframe
def create_excel(shareName,timeframe,limit):  
    data=give(shareName,timeframe,limit)
    df=pd.DataFrame(data,columns=['Open Time','Open','High','Low','Close','Volume',
                                    'Close Time','Quote asset volume','Number of trades','Taker buy base asset volume',
                                    'Taker buy quote asset volume','Ignore'])
    df['Datetime'] = pd.to_datetime(df['Open Time'], unit='ms')
    df.index=df['Datetime']
    columns_to_convert = ['Open','High','Low','Close','Volume']
    df[columns_to_convert] = df[columns_to_convert].astype(float)
    excelName=shareName+timeframe+'.xlsx' # for example: 'BTC1h.xlsx'
    df.to_excel(excelName)
    return df

In [90]:
shareName='ETH'
timeframe='1h'
limit=7*24
prices=create_excel(shareName,timeframe,limit)
prices['fsma']=pta.sma(prices['Close'],5)
prices['ssma']=pta.sma(prices['Close'],20)

In [91]:
# pip install backtesting
from backtesting import Strategy,Backtest
from backtesting.lib import crossover
import numpy as np
class myStrategy(Strategy):
    def init(self):
        def fsma_indicator():
            return self.data['fsma']
        def ssma_indicator():
            return self.data['ssma']
        self.fsma=self.I(fsma_indicator)
        self.ssma=self.I(ssma_indicator)
    def next(self):
        if(crossover(self.fsma,self.ssma)):
            self.buy()
        if(crossover(self.ssma,self.fsma)):
            self.sell()
bt = Backtest(prices, myStrategy, commission=.0004,
                  exclusive_orders=True, margin=0.25)
stats = bt.run()
print(stats)
bt.plot()

Start                     2024-03-22 18:00:00
End                       2024-03-29 17:00:00
Duration                      6 days 23:00:00
Exposure Time [%]                   81.547619
Equity Final [$]                 11485.502792
Equity Peak [$]                  12693.812088
Return [%]                          14.855028
Buy & Hold Return [%]                4.329394
Return (Ann.) [%]                55406.556408
Volatility (Ann.) [%]           143199.990697
Sharpe Ratio                         0.386917
Sortino Ratio                      808.056842
Calmar Ratio                      3117.473094
Max. Drawdown [%]                  -17.772906
Avg. Drawdown [%]                   -6.729157
Max. Drawdown Duration        3 days 08:00:00
Avg. Drawdown Duration        0 days 22:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                        6.04254
Worst Trade [%]                     -1.479557
Avg. Trade [%]                    

Row(id='18994', ...)